In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Proyectos/spotify_analisys/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Proyectos/spotify_analisys


In [2]:
# Instalar BeautifulSoup y requests
!pip install requests beautifulsoup4
!pip install spotipy
!pip install python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 11.4 MB/s eta 0:00:00


In [68]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from dotenv import load_dotenv
import os
from datetime import date

In [54]:
'''
Para poder enriquecer los datos que obtuvimos anteriormente necesitamos
autenticarnos en Spotify para poder obtener mas información de las canciones
'''

load_dotenv()

client_id = os.getenv('SPOTIPY_CLIENT_ID')
client_secret = os.getenv('SPOTIPY_CLIENT_SECRET')

#Autenticación en la API de Spotify
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=client_id,
    client_secret=client_secret
))

In [7]:
# URL del Top 50 Global de Spotify Charts
url = "https://open.spotify.com/playlist/37i9dQZEVXbO3qyFxbkOE1"

# Realizar la solicitud a la página
response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")

    # Aquí extraemos los nombres de las canciones del top 50 de México
    songs = soup.find_all("div", attrs={"aria-label": True})
    song_list = []

    # Extraer e imprimir los valores del atributo aria-label
    for idx, song in enumerate(songs[:50], start=1):  # Limitar a las primeras 10 canciones
        song_list.append(song['aria-label'])
else:
    print(f"Error al acceder a la página: {response.status_code}")


In [70]:
#Lista para almacenar los datos enriquecidos
song_data = []

#Buscar información de cada canción
for song in song_list:
    result = sp.search(q='track:' + song, type='track', limit=1)
    if result['tracks']['items']:
        track = result['tracks']['items'][0]

        #Obtener ID del artista para obtener mas info
        artist_id = track['artists'][0]['id']
        artist_info = sp.artist(artist_id)

        #Obtener género y popularidad del artista
        artist_genres = ", ".join(artist_info['genres'])
        artist_popularity = artist_info['popularity']

        song_data.append({
            #Obtener información básica de la canción
            'song_name': song,
            'artist': track['artists'][0]['name'],
            'album': track['album']['name'],
            'release_date': track['album']['release_date'],
            'popularity': track['popularity'],
            'duration_ms': track['duration_ms'],
            'explicit': track['explicit'],
            'genres': artist_genres,
            'artist_popularity': artist_popularity
        })



#Creación de dataframe con los datos extraídos
df = pd.DataFrame(song_data)
df['load_date'] = date.today()
df.index = [i + 1 for i in range(len(df))]

#Muestra de los primeros registros del DataFrame
df.head().style

,song_name,artist,album,release_date,popularity,duration_ms,explicit,genres,artist_popularity,load_date
1,DtMF,Bad Bunny,DeBÍ TiRAR MáS FOToS,2025-01-05,93,237117,True,"reggaeton, trap latino, urbano latino, latin",100,2025-01-17
2,Loco,Neton Vega,Loco,2024-12-20,85,186521,False,"corrido, corridos tumbados, corridos bélicos, electro corridos",83,2025-01-17
3,ROSONES,Tito Double P,ROSONES,2024-12-05,88,177590,True,"corrido, corridos tumbados, corridos bélicos, electro corridos, música mexicana",90,2025-01-17
4,NADIE,Tito Double P,NADIE,2024-11-21,88,197031,True,"corrido, corridos tumbados, corridos bélicos, electro corridos, música mexicana",90,2025-01-17
5,BAILE INoLVIDABLE,Bad Bunny,DeBÍ TiRAR MáS FOToS,2025-01-05,91,367725,True,"reggaeton, trap latino, urbano latino, latin",100,2025-01-17
